In [5]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
df = pd.read_csv("Features_all.csv")
df = df[(df.modeID != 'audible' )]
df

,channel_1_mav,channel_1_rms,channel_1_iav,channel_1_var,channel_1_ssi,channel_1_var.1,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
50,0.302146,0.428303,855.071838,0.178186,519.144666,0.178186,1256.635712,855.071838,0.444041,872,...,0.444218,874,1624,789,112.765957,117.610824,1,1001,10010201,silent
51,0.300652,0.427435,1234.777893,0.177518,750.350798,0.177518,1823.810242,1234.777893,0.444074,1269,...,0.443859,1297,2357,1125,124.694377,124.815305,1,1001,10010202,silent
52,0.301546,0.427589,724.613983,0.177686,439.345396,0.177686,1066.772095,724.613983,0.443933,714,...,0.442849,720,1374,662,94.142259,110.282368,1,1001,10010203,silent
53,0.301181,0.427158,1138.764557,0.177491,689.897471,0.177491,1681.027466,1138.764557,0.444599,1174,...,0.444029,1144,2160,1056,84.350133,106.473499,1,1001,10010204,silent
54,0.302001,0.428487,1338.771912,0.178612,813.903190,0.178612,1977.487030,1338.771912,0.446083,1356,...,0.445143,1366,2538,1230,120.814480,122.660870,1,1001,10010205,silent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6865,0.207974,0.385068,566.728729,0.137972,404.055786,0.137972,1027.747131,566.728729,0.377155,1677,...,0.375292,1702,2214,389,66.420664,98.458670,8,8019,80190246,whisper
6866,0.206211,0.384257,458.200958,0.137259,328.085320,0.137259,834.120422,458.200958,0.375392,1372,...,0.374308,1374,1823,317,35.294118,89.446197,8,8019,80190247,whisper
6867,0.205649,0.384072,540.651550,0.137091,387.806500,0.137091,982.714752,540.651550,0.373798,1630,...,0.373218,1628,2147,375,44.827586,92.387261,8,8019,80190248,whisper
6868,0.205896,0.384267,411.173523,0.137218,294.878605,0.137218,746.980469,411.173523,0.374051,1245,...,0.374330,1232,1644,285,21.212121,83.388139,8,8019,80190249,whisper


In [9]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data():
    temp_df = df
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_per_user(user_id):
    temp_df = df[(df.userID == user_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

In [14]:
def lda_all_users_separately():
    user_ids = np.unique(df.userID)
    for user_id in user_ids:
        X_train, X_test, y_train, y_test = split_data_per_user(user_id)
        k_clf = LinearDiscriminantAnalysis()
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        print("Accuracy for user " + str(user_id) + " is " + str(k_clf.score(X_test, y_test)))

def lda_all():
    X_train, X_test, y_train, y_test = split_data()
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    print("Accuracy for all users is "+str(k_clf.score(X_test, y_test)))
    
def lda_all_sessions_separately():
    sessionIDs = np.unique(df.sessionID)
    for sessionID in sessionIDs:
        X_train, X_test, y_train, y_test = split_data_per_session(sessionID)
        k_clf = LinearDiscriminantAnalysis()
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        print("Accuracy for session " + str(sessionID) + " is " + str(score))

In [15]:
lda_all_users_separately()

Accuracy for user 1 is 0.6666666666666666
Accuracy for user 2 is 0.72
Accuracy for user 3 is 0.7
Accuracy for user 4 is 0.625
Accuracy for user 5 is 0.7
Accuracy for user 6 is 0.55
Accuracy for user 7 is 0.875
Accuracy for user 8 is 0.6928571428571428


In [16]:
lda_all()

Accuracy for all users is 0.5515625


In [17]:
lda_all_sessions_separately()

Accuracy for session 1001 is 0.7
Accuracy for session 1002 is 0.75
Accuracy for session 1003 is 0.55
Accuracy for session 2001 is 0.75
Accuracy for session 2003 is 0.8
Accuracy for session 2004 is 0.75
Accuracy for session 2005 is 0.85
Accuracy for session 2006 is 0.6
Accuracy for session 2007 is 0.5
Accuracy for session 2008 is 0.8
Accuracy for session 2009 is 0.45
Accuracy for session 2010 is 0.6
Accuracy for session 2012 is 0.4
Accuracy for session 2013 is 0.85
Accuracy for session 2028 is 0.6
Accuracy for session 2029 is 0.6
Accuracy for session 2031 is 0.4
Accuracy for session 2032 is 0.55
Accuracy for session 3001 is 0.75
Accuracy for session 4001 is 0.6
Accuracy for session 4002 is 0.65
Accuracy for session 5001 is 0.5
Accuracy for session 6001 is 0.6
Accuracy for session 7001 is 0.65
Accuracy for session 7002 is 0.55
Accuracy for session 8002 is 0.75
Accuracy for session 8003 is 0.65
Accuracy for session 8010 is 0.7
Accuracy for session 8016 is 0.65
Accuracy for session 8017 is

In [ ]:
#Die Session mit einem Score von 1.0 
#sind Sessions die jediglich 'Audible' audios enthalten

In [69]:
temp_df = df[(df.userID == 2)]

In [70]:
temp_df

,channel_1_mav,channel_1_rms,channel_1_iav,channel_1_var,channel_1_ssi,channel_1_var.1,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
450,0.310488,0.437586,1313.984863,0.186830,810.350225,0.186830,1931.977173,1313.984863,0.456516,1561,...,0.457724,1572,2487,1168,140.758294,138.100889,2,2001,20010100,audible
451,0.309638,0.435938,953.993866,0.185188,585.520489,0.185188,1408.607727,953.993866,0.457192,1166,...,0.452142,1106,1782,859,137.785016,135.651807,2,2001,20010101,audible
452,0.311286,0.436956,1216.195740,0.185732,745.965651,0.185732,1768.965027,1216.195740,0.452768,1408,...,0.455928,1408,2298,1068,144.987147,140.935737,2,2001,20010102,audible
453,0.308020,0.435242,1002.911591,0.184398,616.802571,0.184398,1468.398041,1002.911591,0.450982,1184,...,0.453554,1186,1894,903,140.740741,140.035807,2,2001,20010103,audible
454,0.308642,0.436662,765.124756,0.185522,472.679483,0.185522,1113.729095,765.124756,0.449265,844,...,0.454846,890,1474,679,139.024390,137.873723,2,2001,20010104,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4165,0.311585,0.435739,762.137817,0.184056,464.418388,0.184056,1089.364685,762.137817,0.445366,881,...,0.450230,876,1391,675,127.160494,127.033457,2,2101,21010516,audible
4166,0.313791,0.435628,552.586517,0.184307,334.188554,0.184307,788.655029,552.586517,0.447845,629,...,0.444923,602,985,483,135.428571,131.993641,2,2101,21010517,audible
4167,0.310581,0.432792,522.397644,0.182019,315.053070,0.182019,745.773834,522.397644,0.443385,594,...,0.442866,586,948,462,113.173653,120.240975,2,2101,21010518,audible
4168,0.309613,0.433393,482.686371,0.182240,292.826404,0.182240,702.733337,482.686371,0.450759,581,...,0.445805,551,863,435,118.831169,123.124368,2,2101,21010519,audible
